# Bone suppression(pix2pix)


## 前準備
### 主要パッケージのインポート

In [ ]:
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from IPython.display import display
from logging import basicConfig, getLogger, INFO
basicConfig(level=INFO, format='%(asctime)s %(levelname)s :%(message)s')
logger = getLogger(__name__)

### データディレクトリの指定

In [ ]:
DATA_ROOT = pathlib.Path('Data/Images/chest_xray')
INPUT_IMAGE_DIR = 'bone_enhancement'
TRUTH_IMAGE_DIR = 'bone_suppression'
CLASS_LABELS = ('lung')
IMAGE_EXT = '.png'

### 画像ファイルを基にpd.DataFrameを作成する

In [ ]:
def create_dataset_df(data_root, input_dir, truth_dir, image_ext):
    dfs = []
    root = pathlib.Path(data_root)
    image_filenames = (root / pathlib.Path(input_dir)).glob('*' + image_ext)
    df = pd.DataFrame(image_filenames, columns=['input_path'])
    df['truth_path'] = df['input_path'].map(
        lambda p: root / pathlib.Path(truth_dir) / p.name)
    return df


df_dataset = create_dataset_df(DATA_ROOT, INPUT_IMAGE_DIR, TRUTH_IMAGE_DIR,
                               IMAGE_EXT)
display(df_dataset.head())

### 画像を表示

In [ ]:
N_SAMPLES = 3
plt.figure(figsize=(10, 5))
for i, sample in enumerate(df_dataset.sample(n=N_SAMPLES).itertuples()):
    image = Image.open(sample.input_path)
    truth = Image.open(sample.truth_path)
    plt.figure(figsize=(4, 2))
    plt.subplot(1, 2, 1)
    plt.imshow(image, cmap='gray')
    plt.axis('off')
    plt.subplot(1, 2, 2)
    plt.imshow(truth, cmap='gray')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

## モデル作成

### Generator model
UNet ver.と同じものを使う

In [ ]:
INPUT_SHAPE = [1, 256, 256]
OUTPUT_CHANNELS = 1
UNET_DEPTH=7
IN_CHS = 1
OUT_CHS = 1
INI_CHS = 16
from tut_models import UNet
from torchsummary import summary
summary(UNet(IN_CHS, OUT_CHS, UNET_DEPTH, ini_chs=INI_CHS), INPUT_SHAPE, verbose=0, device='cpu')

### Discriminator model

In [ ]:
import torch
import torch.nn as nn
import tut_models

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()

        self.model = nn.Sequential(
            tut_models.ConvBNReLU(2, 16, 3, 0, leaky=True),
            tut_models.ConvBNReLU(16, 32, 3, 0, leaky=True),
            nn.MaxPool2d(2),
            tut_models.ConvBNReLU(32, 32, 3, 0, leaky=True),
            tut_models.ConvBNReLU(32, 64, 3, 0, leaky=True),
            nn.MaxPool2d(2),
            tut_models.ConvBNReLU(64, 64, 3, 0, leaky=True),
            tut_models.ConvBNReLU(64, 128, 3, 0, leaky=True),
            nn.MaxPool2d(2),
            tut_models.ConvBNReLU(128, 128, 3, 0, leaky=True),
            tut_models.ConvBNReLU(128, 256, 3, 0, leaky=True),
            nn.MaxPool2d(2),
            nn.Conv2d(256, 1, kernel_size=3, padding=0)
        )

    def forward(self, input_image, target_image):
        cat = torch.cat([input_image, target_image], dim=1)
        return self.model(cat)


summary(Discriminator(), [INPUT_SHAPE, INPUT_SHAPE], verbose=0, device='cpu')

## 学習
### ホールドアウト

In [ ]:
from sklearn.model_selection import KFold
N_SPLITS = 5
SEED = 0
kfold = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
train_index, test_index = next(kfold.split(df_dataset['input_path']))

df_train = df_dataset.iloc[train_index]
df_test = df_dataset.iloc[test_index]

In [ ]:
import tqdm
IMG_SHAPE = INPUT_SHAPE


def load_img(filepath):
    img = Image.open(filepath)
    img = img.resize(IMG_SHAPE[1:])
    return np.atleast_3d(img)


def load_segmentation_dataset(df, load_img):
    data = np.stack(
        [load_img(filepath) for filepath in tqdm.tqdm(df['input_path'])])
    labels = np.stack([
        load_img(filepath)[..., :1]
        for filepath in tqdm.tqdm(df['truth_path'])
    ])
    return (data/255).astype(np.float32), (labels/255).astype(np.float32)

train_data, train_labels = load_segmentation_dataset(df_train, load_img)

### Data augmentation

In [ ]:
import albumentations as A

album_transform = A.Compose([
    A.RandomBrightnessContrast(brightness_limit=.1,
                               contrast_limit=.1,
                               p=.5),
    A.HorizontalFlip(p=.25),
    A.ShiftScaleRotate(shift_limit=0, scale_limit=.2, rotate_limit=10, p=.8)
], additional_targets={'image0': 'image'})

def random_transform(x, y):
    tfed = album_transform(image=x, image0=y)
    x, y = tfed['image'], tfed['image0']
    x, y = x.transpose(2,0,1), y.transpose(2,0,1)
    return x.astype(np.float32), y.astype(np.float32)


class AugmentedDataset(torch.utils.data.Dataset):
    def __init__(self, x, y, transform=None):
        self.transform = transform
        self.xs = x
        self.ys = y

    def __len__(self):
        return len(self.xs)

    def __getitem__(self, idx):
        x, y = self.xs[idx], self.ys[idx]
        if self.transform:
            x, y = self.transform(x, y)

        return x, y

N_TEST = 3


def test_augmentation(df_dataset):
    dataset = AugmentedDataset(train_data[:N_TEST], train_labels[:N_TEST], random_transform)

    loader = torch.utils.data.DataLoader(dataset,
                                         batch_size=1,
                                         shuffle=False,
                                         num_workers=0)
    for i, data in enumerate(loader):
        plt.figure(figsize=(4, 1.5))
        plt.subplot(1, 2, 1)
        plt.imshow(data[0].squeeze().numpy(), cmap='gray')
        plt.title('input image')
        plt.axis('off')
        plt.subplot(1, 2, 2)
        plt.imshow(data[1].squeeze().numpy(), cmap='gray')
        plt.title('true image')
        plt.axis('off')
        plt.show()


test_augmentation(df_dataset)

### lit model

In [ ]:
import pytorch_lightning as pl
import torch.optim as optim
import torch.nn.functional as F
from collections import OrderedDict
import matplotlib.pyplot as plt

class LitPix2Pix(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.generator = UNet(IN_CHS, OUT_CHS, UNET_DEPTH, ini_chs=INI_CHS)
        self.discriminator = Discriminator()
        self.pix_weight = 100

    def forward(self, x):
        return self.generator(x)

    def adversarial_loss(self, y_hat, y):
        return F.binary_cross_entropy_with_logits(y_hat, y)
    
    def pixelwise_loss(self, input_data, target_data):
        return F.l1_loss(input_data, target_data)

    def training_step(self, batch, batch_idx, optimizer_idx):
        input_imgs, target_imgs = batch
        self.input = input_imgs
        self.target = target_imgs

        g_out = self(input_imgs)
        self.g_out = g_out
        # train generator
        if optimizer_idx == 0:

            d_out = self.discriminator(input_imgs, self.g_out)

            # adversarial loss is binary cross-entropy
            g_loss_adv = self.adversarial_loss(d_out, torch.ones_like(d_out))
            g_loss_pix = self.pixelwise_loss(g_out, target_imgs)
            g_loss = g_loss_adv + (self.pix_weight * g_loss_pix)
            tqdm_dict = {'g_loss': g_loss}
            output = OrderedDict({
                'loss': g_loss,
                'progress_bar': tqdm_dict,
                'log': tqdm_dict
            })
            return output

        # train discriminator
        if optimizer_idx == 1:
            g_out.detach()

            # target_imgs is real
            d_out = self.discriminator(input_imgs, target_imgs)
            d_loss_real = self.adversarial_loss(d_out, torch.ones_like(d_out))

            # g_out is fake
            d_out = self.discriminator(input_imgs, g_out)
            d_loss_fake = self.adversarial_loss(d_out, torch.zeros_like(d_out))

            # discriminator loss is the average of these
            d_loss = .5 * (d_loss_real + d_loss_fake)
            tqdm_dict = {'d_loss': d_loss}
            output = OrderedDict({
                'loss': d_loss,
                'progress_bar': tqdm_dict,
                'log': tqdm_dict
            })
            return output

    def configure_optimizers(self):
        opt_g = torch.optim.Adam(self.generator.parameters(),
                                 lr=2e-4)
        opt_d = torch.optim.Adam(self.discriminator.parameters(),
                                 lr=2e-4)
        return [opt_g, opt_d], []

    def on_epoch_end(self):
        for i, img in enumerate([self.input[0], self.target[0], self.g_out[0]]):
            img = img.cpu().numpy().squeeze()
            plt.subplot(1,3,i+1)
            plt.imshow(img, cmap='gray')
            plt.axis('off')
        plt.show()

### fit

In [ ]:
import os
EPOCHS = 16
BATCH_SIZE = 4
NUM_WORKERS = 0 if os.name == 'nt' else 2
gpus = 1 if torch.cuda.is_available() else 0
# gpus = 0

model = LitPix2Pix()
trainer = pl.Trainer(gpus=gpus,
                     max_epochs=EPOCHS)

dataset = AugmentedDataset(train_data, train_labels, random_transform)
trainloader = torch.utils.data.DataLoader(dataset,
                                          batch_size=BATCH_SIZE,
                                          shuffle=True,
                                          num_workers=NUM_WORKERS)
trainer.fit(model, trainloader)
logger.info('Finish training')

## 評価
目視

In [ ]:
import math
from sklearn import metrics


def show_images_in_a_row(images, titles, figsize=(15, 5)):
    assert len(images) == len(titles), 'Invalid size of arguments'
    plt.figure(figsize=figsize)
    for i, (image, title) in enumerate(zip(images, titles), 1):
        plt.subplot(1, len(images), i)
        plt.imshow(image, cmap='gray')
        plt.title(title)
        plt.axis('off')
    plt.tight_layout()
    plt.show()

model.eval()
model.freeze()

scores = []
for i, index in enumerate(test_index):
    image = load_img(
        df_dataset.iloc[index].input_path)
    truth = load_img(
        df_dataset.iloc[index].truth_path)
    truth = np.array(truth) / 255
    truth = truth.squeeze()

    image = np.atleast_3d(image)
    image = np.array(image) / 255
    image = image.transpose(2,0,1)
    pred = model(torch.FloatTensor(image[np.newaxis])).squeeze().cpu().numpy()
    image = image.squeeze()
    if i < 5:
        show_images_in_a_row(*zip((image, 'input'), (pred, 'output'), (
            truth, 'truth'), (image - pred, 'input-output'), (
                image - truth, 'input-truth'), (pred - truth, 'output-truth')))
    else:
        break

In [ ]:
trainer.save_checkpoint('pix2pix_generator.ckpt')